In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.4 MB/s 
     |████████████████████████████████| 596 kB 20.8 MB/s 
     |████████████████████████████████| 101 kB 12.7 MB/s 
     |████████████████████████████████| 6.6 MB 9.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, TFAutoModel, AutoConfig
import tensorflow as tf
from sklearn.metrics import f1_score,accuracy_score,precision_score,recall_score
from sklearn.model_selection import StratifiedShuffleSplit


In [ ]:
data=pd.read_csv("/content/aug_dataset.csv")
data["label"]=data["class"].map({"normal":0,"abusive":1,"hate":1})
data["class"].value_counts()

normal     2300
abusive    2028
hate       1724
Name: class, dtype: int64

In [ ]:
def strat_train_test_split(data,target,rate=0.1):
    split=StratifiedShuffleSplit(n_splits=1,test_size=rate,random_state=41)
    for train_index,test_index in split.split(data,data[target]):
        train_set=data.loc[train_index]
        test_set=data.loc[test_index]
    return train_set,test_set

train_set,test_set=strat_train_test_split(data,"label",0.2)
train_set=train_set.reset_index()
train_set,valid_set=strat_train_test_split(train_set,"label",0.1)

In [ ]:
class cfg: 
  pretrained_model="UBC-NLP/MARBERTv2"
  input_shape=128
  learning_rate= 1e-4
  batch_size=32
  epochs=9

In [ ]:
def get_strategy():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experi3mental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        auto = tf.data.experimental.AUTOTUNE
        replicas = strategy.num_replicas_in_sync
        print(f'TPU: {tpu.master()}')
    except:
        strategy = tf.distribute.get_strategy()
        auto = tf.data.experimental.AUTOTUNE
        replicas = strategy.num_replicas_in_sync

    tf.config.optimizer.set_jit(True)
    print(f'Replicas: {replicas}')

    return strategy

strategy=get_strategy()

Replicas: 1


In [ ]:
def tokenize(sentences):
    input_ids, attention_mask = [],[]
    tokenizer=AutoTokenizer.from_pretrained(cfg.pretrained_model)
    for sentence in sentences:
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True,max_length=cfg.input_shape,truncation=True, padding='max_length',return_attention_mask=True, return_token_type_ids=True)
        input_ids.append(inputs['input_ids'])
        attention_mask.append(inputs['attention_mask'])
    return {"input_ids":tf.convert_to_tensor(input_ids),"attention_masks":tf.convert_to_tensor(attention_mask)}

def make_dataset(data):
    inputs=tokenize(data.text.values.astype(str))
    target=data.label
    train = tf.data.Dataset.from_tensor_slices(((inputs["input_ids"],inputs["attention_masks"]),target)).batch(cfg.batch_size)
    return train

train_ds=make_dataset(train_set)
valid_ds=make_dataset(valid_set)
test_ds=make_dataset(test_set)


Downloading:   0%|          | 0.00/439 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def create_model():
    with strategy.scope():
      config =AutoConfig.from_pretrained(cfg.pretrained_model,dropout=0.3,seed=3,attention_dropout=0.3,output_hidden_states = True)
      transformer= TFAutoModel.from_pretrained(cfg.pretrained_model)
      input_ids= tf.keras.layers.Input(shape=(cfg.input_shape,), dtype='int32',name="input_ids")
      input_masks = tf.keras.layers.Input(shape=(cfg.input_shape,), dtype='int32',name="attention_masks")
      embedding_layer=transformer(input_ids, attention_mask=input_masks)[0][:,0,:]
      # embedding_layer=tf.keras.layers.Flatten()(embedding_layer)
      output=tf.keras.layers.Dense(1, activation="sigmoid",name="output")(embedding_layer)
                                                 
      model = tf.keras.Model(inputs=[input_ids, input_masks], outputs = output)
      
    return model                                    

In [ ]:
with strategy.scope():

  model=create_model()
  checkpoint = tf.keras.callbacks.ModelCheckpoint(f'model.h5',monitor='val_loss',mode='min',save_best_only=True,save_weights_only=True,save_freq='epoch',verbose=1)
  model.compile(loss={"output":tf.keras.losses.BinaryCrossentropy()},
                      optimizer=tf.keras.optimizers.Adam(cfg.learning_rate))
  
  model.fit(train_ds,epochs=cfg.epochs,verbose=1,validation_data = valid_ds,callbacks = [checkpoint])


Downloading:   0%|          | 0.00/757 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/621M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at UBC-NLP/MARBERTv2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/9
137/137 [==============================] - ETA: 0s - loss: 0.4767
Epoch 1: val_loss improved from inf to 0.37841, saving model to model.h5
137/137 [==============================] - 269s 2s/step - loss: 0.4767 - val_loss: 0.3784
Epoch 2/9
137/137 [==============================] - ETA: 0s - loss: 0.2368
Epoch 2: val_loss did not improve from 0.37841
137/137 [==============================] - 112s 817ms/step - loss: 0.2368 - val_loss: 0.6253
Epoch 3/9
137/137 [==============================] - ETA: 0s - loss: 0.1418
Epoch 3: val_loss did not improve from 0.37841
137/137 [==============================] - 102s 743ms/step - loss: 0.1418 - val_loss: 0.5367
Epoch 4/9
137/137 [==============================] - ETA: 0s - loss: 0.0936
Epoch 4: val_loss did not improve from 0.37841
137/137 [==============================] - 102s 742ms/step - loss: 0.0936 - val_loss: 0.8656
Epoch 5/9
137/137 [==============================] - ETA: 0s - loss: 0.0416
Epoch 5: val_loss did not improve fro

In [ ]:
def report_gen(predictions,labels):
    report={
    "F1_macro":f1_score(predictions,labels,average="macro"),
    "Accuracy":accuracy_score(predictions,labels),
    "Precision_macro":precision_score(predictions,labels,average="macro"),
    "Recall_macro":recall_score(predictions,labels,average="macro")
    } 
    return report

predictions=model.predict(test_ds)
predictions=[int(i>0.7) for i in predictions]
print(report_gen(predictions,test_set["label"]))

{'F1_macro': 0.7961028731391359, 'Accuracy': 0.8100743187448389, 'Precision_macro': 0.7929601111561397, 'Recall_macro': 0.8001203968945905}
